# Part I. ETL Pipeline for Pre-Processing the Files

## CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
  
# join the file path and roots with the subdirectories using glob
file_path_list = glob.glob(filepath+'/*.csv')

len(file_path_list)

/home/workspace


30

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty dataframe that will hold all rows generated from each file
events = pd.DataFrame(columns=['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',\
       'level', 'location', 'sessionId', 'song', 'userId']) 

# for every filepath in the file path list 
for f in file_path_list:
    # read in the csv file as a dataframe and append to bigger frame only columns we want
    df = pd.read_csv(f)[list(events.columns)]
    events = events.append(df, ignore_index=True)
            
#without null values
events = events.dropna()
#save as the file
events.to_csv('event_datafile_new.csv')
events.shape

(6820, 11)

In [4]:
# check df format
events.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88.0
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88.0


I noticed that the `user_id` field is in the form of a float, so cleaning that up to have it be an integer before adding it to any keyspace.

In [5]:
#clean up user ID to be int
events.userId = events.userId.apply(lambda x: int(x))
events.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88


# Part II. Complete the Apache Cassandra ETL Process. 

## A CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory is generated from the previous code. It contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [6]:
events.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

## Creating the Apache Cassandra Keyspace


#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

This function will allow me to run all queries without replicating this chunk of code for everytime we create, insert into, or select from a table.

In [10]:
#method for running query and giving results
def exec_query(session, query, select=False):
    rows = None
    try:
        if select==True:
            rows = session.execute(query)
        else:
            session.execute(query)
    except Exception as e:
        print(e)
    return rows

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


Before inserting the data and reading it from the cassandra keyspace, I decided to see what the result of each query would be by recreating the selections in the pandas dataframe. This allows me to have a quality check when I get the results from my query to make sure I'm making the correct selections.

In [11]:
#Question 1:
events[(events.sessionId == 338) & (events.itemInSession == 4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
546,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [12]:
#Question 2:
events[(events.userId == 10) & (events.sessionId == 182)][['artist', 'song', 'firstName', 'lastName', 'itemInSession', 'userId', 'sessionId']]

,artist,song,firstName,lastName,itemInSession,userId,sessionId
5506,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0,10,182
5507,Three Drives,Greece 2000,Sylvie,Cruz,1,10,182
5509,Sebastien Tellier,Kilometer,Sylvie,Cruz,2,10,182
5510,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3,10,182


In [13]:
#Question 3:
events[events.song=='All Hands Against His Own'][['firstName', 'lastName']]

,firstName,lastName
3286,Tegan,Levine
6012,Sara,Johnson
7391,Jacqueline,Lynch


**Query 1:**  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [14]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_query = 'CREATE TABLE IF NOT EXISTS song_sessions '
pk = 'PRIMARY KEY(sessionId, itemInSession)'
create_query = create_query + '(sessionId int, itemInSession int, artist varchar, song varchar, length varchar, {})'.format(pk)
exec_query(session, 'DROP TABLE IF EXISTS song_sessions')
exec_query(session, create_query)

In [15]:
# Add rows from dataframe to DB
for i, row in events.iterrows():
##Assign the INSERT statements into the `query` variable
    query = "INSERT INTO song_sessions (sessionId, itemInSession, artist, song, length) "
    #have an issue with punctuation on artist
    art_song = [row['artist'], row['song']]
    #create a quote escape for song and artist strings before adding to table
    art_song = [j.split("'")[0]+'\'\''+ j.split("'")[1] if j.find("'") != -1 else j for j in art_song]
    query = query + "VALUES({},{},'{}','{}','{}')".format(row.sessionId, row.itemInSession, art_song[0], art_song[1], str(row.length))
    exec_query(session, query)

Query to select the artist, song, and length of the song for a specific sessionId 338 with itemInSession of 4

In [16]:
##Add in the SELECT statement to verify the data was entered into the table
select_query = 'SELECT artist, song, length FROM song_sessions WHERE sessionId = 338 AND itemInSession = 4'
results = session.execute(select_query)
display = []
for row in results:
    display.append([row.artist, row.song, row.length])
pd.DataFrame(display, columns=['artist', 'song', 'length'])

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


**Query 2:** Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [17]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_query = 'CREATE TABLE IF NOT EXISTS artist_session_by_songs '
pk = 'PRIMARY KEY((userId, sessionid), itemInSession)'
create_query = create_query + '(userId int, sessionId int, itemInSession int, artist varchar, song varchar, firstName varchar, lastName varchar, {})'.format(pk)
exec_query(session, 'DROP TABLE IF EXISTS artist_session_by_songs')
exec_query(session, create_query)

In [18]:
# Add rows from dataframe to DB
for i, row in events.iterrows():
## Assign the INSERT statements into the `query` variable
    query = "INSERT INTO artist_session_by_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
    #have an issue with punctuation on artist
    data = [row['artist'], row['song'], row['firstName'], row['lastName']]
    #create a quote escape for song and artist strings before adding to table
    data = [j.split("'")[0]+'\'\''+ j.split("'")[1] if j.find("'") != -1 else j for j in data]
    query = query + "VALUES({},{},{},'{}','{}','{}','{}')".format(row.userId, row.sessionId, row.itemInSession, data[0], data[1], data[2], data[3])
    exec_query(session, query)

Selection of artist, song, and the user's first and last name using their user ID during a specific session

In [19]:
## Add in the SELECT statement to verify the data was entered into the table
select_query = 'SELECT artist, song, firstName, lastName FROM artist_session_by_songs WHERE userId = 10 AND sessionId = 182'
results = session.execute(select_query)
display = []
for row in results:
    display.append([row.artist, row.song, row.firstname, row.lastname])
pd.DataFrame(display, columns=['artist', 'song', 'first_name', 'last_name'])

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


**Query 3:** Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [20]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_query = 'CREATE TABLE IF NOT EXISTS user_session_by_artists '
# someone may listen to multiple songs during one session hence adding user and session ids as clustering cols
pk = 'PRIMARY KEY(song, userId, sessionId)'
create_query = create_query + '(song varchar, userId int, sessionId int, firstName varchar, lastName varchar, {})'.format(pk)
exec_query(session, 'DROP TABLE IF EXISTS user_session_by_artists')
exec_query(session, create_query)                    

In [21]:
# Add rows from dataframe to DB
for i, row in events.iterrows():
## Assign the INSERT statements into the `query` variable
    query = "INSERT INTO user_session_by_artists (song, userId, sessionId, firstName, lastName) "
    #have an issue with punctuation on artist
    data = [row['song'], row['firstName'], row['lastName']]
    #create a quote escape for song and artist strings before adding to table
    data = [j.split("'")[0]+'\'\''+ j.split("'")[1] if j.find("'") != -1 else j for j in data]
    query = query + "VALUES('{}',{},{},'{}','{}')".format(data[0], row.userId, row.sessionId, data[1], data[2])
    exec_query(session, query)

Providing the user's first and last names for those who listened to a specific song

In [22]:
## Add in the SELECT statement to verify the data was entered into the table
select_query = "SELECT firstName, lastName FROM user_session_by_artists WHERE song = 'All Hands Against His Own'"
results = session.execute(select_query)
display = []
for row in results:
    #combine first and last names to get user name
    display.append([row.firstname + ' ' + row.lastname])
pd.DataFrame(display, columns=['user_name'])

,user_name
0,Jacqueline Lynch
1,Tegan Levine
2,Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
exec_query(session, 'DROP TABLE IF EXISTS song_sessions')
exec_query(session, 'DROP TABLE IF EXISTS artist_session_by_songs')
exec_query(session, 'DROP TABLE IF EXISTS user_session_by_artists')

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()